# Designed to Clean / Standardize any Dataset into a standard format

In [351]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import calendar
import string
from string import punctuation
from itertools import chain

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_selection import SelectPercentile, chi2, f_regression, f_classif

from sklearn import svm
from sklearn.utils import shuffle

df = pd.read_csv('Tunapuna-Piarco_weather_data_2000_2025.csv', encoding='latin1')  #load Datafield csv
pd.options.mode.copy_on_write = True 


# Read and Understand the Dataset

In [353]:
df.shape

(9215, 28)

In [354]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9215 entries, 0 to 9214
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              9215 non-null   object 
 1   datetime          9215 non-null   object 
 2   tempmax           9215 non-null   float64
 3   tempmin           9215 non-null   float64
 4   temp              9215 non-null   float64
 5   feelslikemax      9215 non-null   float64
 6   feelslikemin      9215 non-null   float64
 7   feelslike         9215 non-null   float64
 8   dew               9215 non-null   float64
 9   humidity          9215 non-null   float64
 10  precip            7419 non-null   float64
 11  precipprob        9215 non-null   float64
 12  precipcover       9215 non-null   float64
 13  windspeed         9215 non-null   float64
 14  winddir           9185 non-null   float64
 15  sealevelpressure  8933 non-null   float64
 16  cloudcover        9215 non-null   float64


In [355]:
df.head(10)   #shows the first 10 rows of the CSV to highlight what the data looks like


,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarradiation,solarenergy,uvindex,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",01/01/2000,87.9,71.7,80.0,94.4,71.7,82.7,72.0,78.0,...,NaN,NaN,NaN,2000-01-01T06:23:15,2000-01-01T17:54:11,0.83,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,78970099999
1,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",02/01/2000,87.9,71.3,78.2,96.1,71.3,80.5,72.5,83.7,...,NaN,NaN,NaN,2000-01-02T06:23:39,2000-01-02T17:54:43,0.86,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,78970099999
2,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",03/01/2000,89.7,73.5,79.3,95.0,73.5,81.2,71.6,78.6,...,NaN,NaN,NaN,2000-01-03T06:24:02,2000-01-03T17:55:16,0.90,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,78970099999
3,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",04/01/2000,84.3,71.3,76.1,89.8,71.3,77.2,70.9,84.9,...,NaN,NaN,NaN,2000-01-04T06:24:24,2000-01-04T17:55:49,0.93,Partially cloudy,Clearing in the afternoon.,partly-cloudy-day,78970099999
4,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",05/01/2000,81.2,73.1,76.1,86.7,73.1,76.5,73.9,93.0,...,NaN,NaN,NaN,2000-01-05T06:24:46,2000-01-05T17:56:21,0.96,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,78970099999
5,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",06/01/2000,87.9,73.5,77.9,96.1,73.5,79.7,73.4,86.7,...,NaN,NaN,NaN,2000-01-06T06:25:07,2000-01-06T17:56:54,0.00,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,78970099999
6,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",07/01/2000,87.5,73.5,77.3,92.9,73.5,78.4,71.5,83.4,...,NaN,NaN,NaN,2000-01-07T06:25:27,2000-01-07T17:57:27,0.03,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,78970099999
7,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",08/01/2000,87.9,72.8,79.7,92.9,72.8,81.6,70.6,75.8,...,NaN,NaN,NaN,2000-01-08T06:25:46,2000-01-08T17:57:59,0.07,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,78970099999
8,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",09/01/2000,87.9,71.7,78.5,95.4,71.7,81.3,72.3,82.3,...,NaN,NaN,NaN,2000-01-09T06:26:05,2000-01-09T17:58:31,0.10,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,78970099999
9,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",10/01/2000,88.4,73.5,79.6,95.7,73.5,82.0,71.9,78.8,...,NaN,NaN,NaN,2000-01-10T06:26:22,2000-01-10T17:59:03,0.14,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,78970099999


In [356]:
df.describe()

,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,moonphase
count,9215.000000,9215.000000,9215.000000,9215.000000,9215.000000,9215.000000,9215.000000,9215.000000,7419.000000,9215.000000,9215.000000,9215.000000,9185.000000,8933.000000,9215.000000,9212.00000,5596.000000,5596.000000,5596.000000,9215.000000
mean,88.839371,74.790613,80.669680,97.387412,74.802073,84.071232,73.382583,79.948736,0.185958,46.337493,4.938359,15.197819,95.908666,1012.596138,59.140912,7.26688,218.167084,18.842959,7.670836,0.483277
std,2.952612,2.286676,1.980641,5.226385,2.328441,3.539485,2.456164,6.157379,0.443432,49.868386,12.599518,4.956746,31.949204,1.529507,17.733937,1.07339,55.505259,4.792457,1.666989,0.288550
min,71.700000,37.300000,71.000000,71.700000,37.300000,71.000000,62.200000,57.200000,0.000000,0.000000,0.000000,0.000000,0.100000,1006.000000,15.300000,2.70000,0.000000,0.000000,0.000000,0.000000
25%,87.700000,73.400000,79.300000,93.700000,73.400000,81.400000,71.700000,75.500000,0.000000,0.000000,0.000000,12.800000,76.500000,1011.600000,44.600000,6.60000,183.700000,15.900000,7.000000,0.250000
50%,89.600000,75.200000,80.700000,97.600000,75.200000,84.000000,73.900000,79.600000,0.016000,0.000000,0.000000,15.000000,96.500000,1012.700000,56.300000,7.20000,227.850000,19.700000,8.000000,0.500000
75%,91.300000,76.500000,82.200000,100.600000,76.500000,86.600000,75.300000,84.400000,0.161000,100.000000,4.170000,17.200000,108.000000,1013.700000,74.200000,7.80000,261.325000,22.600000,9.000000,0.750000
max,138.100000,80.600000,86.200000,138.100000,88.300000,96.800000,79.100000,99.300000,9.134000,100.000000,100.000000,162.400000,360.000000,1017.600000,100.000000,16.80000,328.700000,28.200000,10.000000,0.980000


In [357]:
df.isna()   # returns all Null values, where Null = true.

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarradiation,solarenergy,uvindex,sunrise,sunset,moonphase,conditions,description,icon,stations
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9210,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9211,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9212,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9213,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [358]:
df.isna().sum()

name                   0
datetime               0
tempmax                0
tempmin                0
temp                   0
feelslikemax           0
feelslikemin           0
feelslike              0
dew                    0
humidity               0
precip              1796
precipprob             0
precipcover            0
windspeed              0
winddir               30
sealevelpressure     282
cloudcover             0
visibility             3
solarradiation      3619
solarenergy         3619
uvindex             3619
sunrise                0
sunset                 0
moonphase              0
conditions             0
description            0
icon                   0
stations               0
dtype: int64

In [359]:
for col in df.columns:
    print(col,df[col].nunique())

name 1
datetime 9215
tempmax 150
tempmin 109
temp 123
feelslikemax 300
feelslikemin 119
feelslike 219
dew 147
humidity 344
precip 659
precipprob 2
precipcover 25
windspeed 133
winddir 1196
sealevelpressure 102
cloudcover 713
visibility 105
solarradiation 1985
solarenergy 241
uvindex 11
sunrise 9215
sunset 9215
moonphase 96
conditions 6
description 33
icon 5
stations 20


# Basic Data Preprocessing

Sunrise / sunset data convereted to just only contain time in 24 hour format.
Icon / Station / wind direction / sea pressure data was removed as deemed unncessary.
Tempratures converted to C from F.
Onehat Encoding for Condition.
Season Categroy added, converting it to Wet and Dry.
Precip converted from Inches to MM.
name truncated to only location (example port of spain)


In [361]:
df.apply(pd.isnull).sum()/df.shape[0]

name                0.000000
datetime            0.000000
tempmax             0.000000
tempmin             0.000000
temp                0.000000
feelslikemax        0.000000
feelslikemin        0.000000
feelslike           0.000000
dew                 0.000000
humidity            0.000000
precip              0.194900
precipprob          0.000000
precipcover         0.000000
windspeed           0.000000
winddir             0.003256
sealevelpressure    0.030602
cloudcover          0.000000
visibility          0.000326
solarradiation      0.392729
solarenergy         0.392729
uvindex             0.392729
sunrise             0.000000
sunset              0.000000
moonphase           0.000000
conditions          0.000000
description         0.000000
icon                0.000000
stations            0.000000
dtype: float64

In [362]:
df["precip"].value_counts() / df.shape[0]

precip
0.000    0.341725
0.039    0.026913
0.079    0.024091
0.012    0.023440
0.020    0.020727
           ...   
0.160    0.000109
0.195    0.000109
0.140    0.000109
0.316    0.000109
3.543    0.000109
Name: count, Length: 659, dtype: float64

In [363]:
df.drop(columns=['icon'],inplace=True)
df.drop(columns=['stations'],inplace=True)
df.drop(columns=['winddir'],inplace=True)
df.drop(columns=['sealevelpressure'],inplace=True)
df.drop(columns=['description'],inplace=True)
df.drop(columns=['conditions'],inplace=True)
df.drop(columns=['solarradiation'],inplace=True)
df.drop(columns=['solarenergy'],inplace=True)
df.drop(columns=['uvindex'],inplace=True)
df.drop(columns=['moonphase'],inplace=True)
df.drop(columns=['sunrise'],inplace=True)
df.drop(columns=['sunset'],inplace=True)
df.drop(columns=['precipcover'], inplace=True)

df.head(10)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,windspeed,cloudcover,visibility
0,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",01/01/2000,87.9,71.7,80.0,94.4,71.7,82.7,72.0,78.0,NaN,0.0,15.0,43.8,7.6
1,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",02/01/2000,87.9,71.3,78.2,96.1,71.3,80.5,72.5,83.7,NaN,0.0,16.1,57.9,8.2
2,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",03/01/2000,89.7,73.5,79.3,95.0,73.5,81.2,71.6,78.6,NaN,0.0,12.8,73.2,9.5
3,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",04/01/2000,84.3,71.3,76.1,89.8,71.3,77.2,70.9,84.9,NaN,0.0,13.9,77.7,9.7
4,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",05/01/2000,81.2,73.1,76.1,86.7,73.1,76.5,73.9,93.0,NaN,0.0,11.4,76.0,8.2
5,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",06/01/2000,87.9,73.5,77.9,96.1,73.5,79.7,73.4,86.7,NaN,0.0,18.3,80.0,6.8
6,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",07/01/2000,87.5,73.5,77.3,92.9,73.5,78.4,71.5,83.4,NaN,0.0,15.0,64.2,7.4
7,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",08/01/2000,87.9,72.8,79.7,92.9,72.8,81.6,70.6,75.8,NaN,0.0,13.9,42.2,8.4
8,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",09/01/2000,87.9,71.7,78.5,95.4,71.7,81.3,72.3,82.3,NaN,0.0,16.1,51.4,7.1
9,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",10/01/2000,88.4,73.5,79.6,95.7,73.5,82.0,71.9,78.8,NaN,0.0,18.3,42.4,8.5


Convert rain to mm

In [365]:
def inches_to_mm(inches):
    return inches * 25.4

df['precip'] = df['precip'].apply(lambda x: pd.Series(inches_to_mm(x)))
df.rename(columns={"windspeed": "windspeed mph"})

df.head(10)


,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,windspeed,cloudcover,visibility
0,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",01/01/2000,87.9,71.7,80.0,94.4,71.7,82.7,72.0,78.0,NaN,0.0,15.0,43.8,7.6
1,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",02/01/2000,87.9,71.3,78.2,96.1,71.3,80.5,72.5,83.7,NaN,0.0,16.1,57.9,8.2
2,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",03/01/2000,89.7,73.5,79.3,95.0,73.5,81.2,71.6,78.6,NaN,0.0,12.8,73.2,9.5
3,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",04/01/2000,84.3,71.3,76.1,89.8,71.3,77.2,70.9,84.9,NaN,0.0,13.9,77.7,9.7
4,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",05/01/2000,81.2,73.1,76.1,86.7,73.1,76.5,73.9,93.0,NaN,0.0,11.4,76.0,8.2
5,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",06/01/2000,87.9,73.5,77.9,96.1,73.5,79.7,73.4,86.7,NaN,0.0,18.3,80.0,6.8
6,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",07/01/2000,87.5,73.5,77.3,92.9,73.5,78.4,71.5,83.4,NaN,0.0,15.0,64.2,7.4
7,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",08/01/2000,87.9,72.8,79.7,92.9,72.8,81.6,70.6,75.8,NaN,0.0,13.9,42.2,8.4
8,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",09/01/2000,87.9,71.7,78.5,95.4,71.7,81.3,72.3,82.3,NaN,0.0,16.1,51.4,7.1
9,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",10/01/2000,88.4,73.5,79.6,95.7,73.5,82.0,71.9,78.8,NaN,0.0,18.3,42.4,8.5


Convert Degrees


In [367]:
def fahrenheit_to_celsius(fahrenheit):
    celsius = (fahrenheit - 32) * 5 / 9
    return round(celsius, 1)


df[[ 'tempmax c']] = df ['tempmax'].apply(lambda x: pd.Series(fahrenheit_to_celsius(x)))
df[[ 'tempmin c']] = df ['tempmin'].apply(lambda x: pd.Series(fahrenheit_to_celsius(x)))
df[[ 'avgtemp c']] = df ['temp'].apply(lambda x: pd.Series(fahrenheit_to_celsius(x)))

df[[ 'feelslikemax c']] = df ['feelslikemax'].apply(lambda x: pd.Series(fahrenheit_to_celsius(x)))
df[[ 'feelslikemin c']] = df ['feelslikemin'].apply(lambda x: pd.Series(fahrenheit_to_celsius(x)))
df[[ 'avgfeelsliketemp c']] = df ['feelslike'].apply(lambda x: pd.Series(fahrenheit_to_celsius(x)))

df[[ 'dewpoint c']] = df ['dew'].apply(lambda x: pd.Series(fahrenheit_to_celsius(x)))

df.drop(columns=['temp'],inplace=True)
df.drop(columns=['tempmax'],inplace=True)
df.drop(columns=['tempmin'],inplace=True)

df.drop(columns=['feelslike'],inplace=True)
df.drop(columns=['feelslikemax'],inplace=True)
df.drop(columns=['feelslikemin'],inplace=True)
df.drop(columns=['dew'],inplace=True)

df.head(10)

,name,datetime,humidity,precip,precipprob,windspeed,cloudcover,visibility,tempmax c,tempmin c,avgtemp c,feelslikemax c,feelslikemin c,avgfeelsliketemp c,dewpoint c
0,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",01/01/2000,78.0,NaN,0.0,15.0,43.8,7.6,31.1,22.1,26.7,34.7,22.1,28.2,22.2
1,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",02/01/2000,83.7,NaN,0.0,16.1,57.9,8.2,31.1,21.8,25.7,35.6,21.8,26.9,22.5
2,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",03/01/2000,78.6,NaN,0.0,12.8,73.2,9.5,32.1,23.1,26.3,35.0,23.1,27.3,22.0
3,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",04/01/2000,84.9,NaN,0.0,13.9,77.7,9.7,29.1,21.8,24.5,32.1,21.8,25.1,21.6
4,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",05/01/2000,93.0,NaN,0.0,11.4,76.0,8.2,27.3,22.8,24.5,30.4,22.8,24.7,23.3
5,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",06/01/2000,86.7,NaN,0.0,18.3,80.0,6.8,31.1,23.1,25.5,35.6,23.1,26.5,23.0
6,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",07/01/2000,83.4,NaN,0.0,15.0,64.2,7.4,30.8,23.1,25.2,33.8,23.1,25.8,21.9
7,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",08/01/2000,75.8,NaN,0.0,13.9,42.2,8.4,31.1,22.7,26.5,33.8,22.7,27.6,21.4
8,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",09/01/2000,82.3,NaN,0.0,16.1,51.4,7.1,31.1,22.1,25.8,35.2,22.1,27.4,22.4
9,"Piarco, Tunapunas-Piarco, Trinidad, Trinidad A...",10/01/2000,78.8,NaN,0.0,18.3,42.4,8.5,31.3,23.1,26.4,35.4,23.1,27.8,22.2


In [368]:
df = df.fillna(method="ffill")

C:\Users\cramd\AppData\Local\Temp\ipykernel_36560\567689999.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")


In [369]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9215 entries, 0 to 9214
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                9215 non-null   object 
 1   datetime            9215 non-null   object 
 2   humidity            9215 non-null   float64
 3   precip              7419 non-null   float64
 4   precipprob          9215 non-null   float64
 5   windspeed           9215 non-null   float64
 6   cloudcover          9215 non-null   float64
 7   visibility          9215 non-null   float64
 8   tempmax c           9215 non-null   float64
 9   tempmin c           9215 non-null   float64
 10  avgtemp c           9215 non-null   float64
 11  feelslikemax c      9215 non-null   float64
 12  feelslikemin c      9215 non-null   float64
 13  avgfeelsliketemp c  9215 non-null   float64
 14  dewpoint c          9215 non-null   float64
dtypes: float64(13), object(2)
memory usage: 1.1+ MB


In [370]:
for col in df.columns:
    print(col,df[col].nunique())

name 1
datetime 9215
humidity 344
precip 659
precipprob 2
windspeed 133
cloudcover 713
visibility 105
tempmax c 103
tempmin c 75
avgtemp c 72
feelslikemax c 185
feelslikemin c 82
avgfeelsliketemp c 127
dewpoint c 87


In [371]:
df.isna().sum()

name                     0
datetime                 0
humidity                 0
precip                1796
precipprob               0
windspeed                0
cloudcover               0
visibility               0
tempmax c                0
tempmin c                0
avgtemp c                0
feelslikemax c           0
feelslikemin c           0
avgfeelsliketemp c       0
dewpoint c               0
dtype: int64

# Outlier Temprature Analysis
Max Temp Ever Recorded in Triniad : 37.8
Lowest Temp Ever Recorded in Trinidad : 16.1

https://www.metoffice.gov.tt/Climate#:~:text=The%20coolest%20minimum%20temperature%20is,and%2030th%2C%201964%20at%20Piarco.

Backfill null values with avg for that day to avoid squewing data

In [374]:
df["datetime"] = pd.to_datetime(df["datetime"], format='mixed' ,dayfirst=True)
df["day"] = df["datetime"].dt.day
df["month"] = df["datetime"].dt.month
df["year"] = df["datetime"].dt.year

ref_df = df.loc['01/0/2005':'31/12/2025']

day_avg_precip = (ref_df[ref_df['precip'].notna()].groupby(['month', 'day'])['precip'].mean())
missing_mask = (df.year <= 2004) & (df['precip'].isna())
df.loc[missing_mask, 'precip'] = df[missing_mask].apply(lambda row: day_avg_precip.get((row['month'], row['day']), None),axis=1)

In [375]:
df.drop(columns=['precipprob'], inplace=True)
df['precip_chance'] = (df['precip'] > 0.00).astype(int)

Replace outliers with more than 36C actual temp with average for that day

In [377]:
day_avg_tempmax = df[df['tempmax c'] <= 37.8].groupby(['month', 'day'])['tempmax c'].mean()

mask = df['tempmax c'] > 37.8

df.loc[mask, 'tempmax c'] = df[mask].apply(
    lambda row: day_avg_tempmax.get((row['month'], row['day']), row['tempmax c']),
    axis=1
)

Replace outlier mintemp < 16.1 with average for that day

In [379]:
day_avg_tempmin = df[df['tempmin c'] >= 16.1].groupby(['month', 'day'])['tempmin c'].mean()

mask = df['tempmin c'] < 16.1

df.loc[mask, 'tempmin c'] = df[mask].apply(
    lambda row: day_avg_tempmin.get((row['month'], row['day']), row['tempmin c']),
    axis=1
)

# Replace outlier feelsmintemp < 16.1 with average for that day

In [381]:
day_avg_tempminfeellike = df[df['feelslikemin c'] >= 16.1].groupby(['month', 'day'])['feelslikemin c'].mean()

mask = df['feelslikemin c'] < 16.1

df.loc[mask, 'feelslikemin c'] = df[mask].apply(
    lambda row: day_avg_tempminfeellike.get((row['month'], row['day']), row['feelslikemin c']),
    axis=1
)
df.drop(columns=['month', 'day', 'year'], inplace=True)


In [382]:
# Ensure 'datetime' column is a datetime type
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

# Filter out rows before 2005
df = df[df['datetime'] >= '2005-01-01']


In [383]:
def rename_all_columns():
    df['name'] = df['name'].apply(lambda x: x.split(',')[0].strip())
    location = df['name'].iloc[0]

    return location

# Run function and save cleaned file
location = rename_all_columns()
fname = f'{location} cleaned.csv'
df.to_csv(fname, index=False)
